In [1]:
%%capture
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import math

import seaborn as sns
import matplotlib.pyplot as plt

from py_helper_functions import *

from datetime import datetime

from patsy.highlevel import dmatrices

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.inspection import permutation_importance
from sklearn.inspection import PartialDependenceDisplay
from sklearn.inspection import partial_dependence
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import ElasticNet

In [71]:
def get_comp_default(df):
    # add all missing year and comp_id combinations -
    # originally missing combinations will have NAs in all other columns
    df = (
        df.set_index(["year", "comp_id"])
        .unstack(fill_value="toReplace")
        .stack()
        .reset_index()
    )
    df = df.replace("toReplace", np.nan)  # only way I could define it as NaN
    
    # generate status_alive; if sales larger than zero and not-NA, then firm is alive
    df["status_alive"] = (df["sales"] > 0 & (False == df["sales"].isna())).astype(int)
    
    # defaults in one year if there are sales in this year but no sales one year later
    # Status_in_one_years: data.groupby('comp_id')['status_alive'].shift(-1)
    df["default"] = (
        (df["status_alive"] == 1)
        & (df.groupby("comp_id")["status_alive"].shift(-1) == 0)
    ).astype(int)
    
    return df

def get_cleaned_data(local=True) -> pd.DataFrame:
    '''
    This function reads from csv files, cleans it and returns the cleaned dataframe
    :param local: default True
    :param src: path to file
    :return: dataframe with cleaned data
    '''
    
    if local:
        raw_files = ['cs_bisnode_panel1.csv', 'cs_bisnode_panel2.csv']
    else:
        raw_files = ['https://raw.githubusercontent.com/viethngn/Data_Analysis_3_ECBS5171/main/assignment3/cs_bisnode_panel1.csv', 'https://raw.githubusercontent.com/viethngn/Data_Analysis_3_ECBS5171/main/assignment3/cs_bisnode_panel2.csv']
        
    dfs = []
    for file in raw_files:
        u_df = pd.read_csv(file)
        dfs.append(u_df)
    df = pd.concat(dfs, ignore_index=True)
    
    # drop variables with many NAs
    df = df.drop(columns=["COGS", "finished_prod", "net_dom_sales", "net_exp_sales", "wages", "D", "exit_year", "exit_date", "birth_year", "labor_avg", "founded_year"])

    working_sample = get_comp_default(df[(df.ind2 == 26)])
    
    del df
        
    return working_sample[(working_sample["begin"].notna()) & (work_df.year < 2016)]

In [72]:
work_df = get_cleaned_data()
# work_df = get_cleaned_data(local=False)
work_df.head()

,year,comp_id,begin,end,amort,curr_assets,curr_liab,extra_exp,extra_inc,extra_profit_loss,...,origin,nace_main,ind2,ind,urban_m,region_m,founded_date,labor_avg,status_alive,default
0,2005,6538183.0,01/01/2005,31/12/2005,792.592590,6237.037109,348.148163,0.000000,0.000000,0.000000,...,Foreign,2630.0,26.0,2.0,2.0,East,25/08/1992,NaN,1,0
1,2005,6934257.0,13/05/2005,31/12/2005,803.703674,4648.147949,9311.111328,0.000000,0.000000,0.000000,...,NaN,2660.0,26.0,2.0,3.0,East,26/04/2005,NaN,1,1
2,2005,8416055.0,01/01/2005,31/12/2005,3155.555664,71070.367190,25514.814450,74.074074,0.000000,-74.074074,...,Domestic,2651.0,26.0,2.0,1.0,Central,28/08/1995,NaN,1,0
5,2005,12428378.0,01/01/2005,31/12/2005,3137.037109,1740.740723,2192.592529,0.000000,0.000000,0.000000,...,Domestic,2660.0,26.0,2.0,2.0,Central,27/06/1991,NaN,1,0
7,2005,15711554.0,01/01/2005,31/12/2005,1592.592651,4740.740723,4970.370605,0.000000,7407.407227,7407.407227,...,Domestic,2630.0,26.0,2.0,1.0,Central,01/09/2001,NaN,1,0


In [86]:
work_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14289 entries, 0 to 20118
Data columns (total 42 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   year                  14289 non-null  int64  
 1   comp_id               14289 non-null  float64
 2   begin                 14289 non-null  object 
 3   end                   14289 non-null  object 
 4   amort                 14003 non-null  float64
 5   curr_assets           14284 non-null  float64
 6   curr_liab             14284 non-null  float64
 7   extra_exp             14007 non-null  float64
 8   extra_inc             14007 non-null  float64
 9   extra_profit_loss     14056 non-null  float64
 10  fixed_assets          14284 non-null  float64
 11  inc_bef_tax           14056 non-null  float64
 12  intang_assets         14284 non-null  float64
 13  inventories           14284 non-null  float64
 14  liq_assets            14284 non-null  float64
 15  material_exp          14

In [85]:
work_df[(work_df.origin.isna())]

,year,comp_id,begin,end,amort,curr_assets,curr_liab,extra_exp,extra_inc,extra_profit_loss,...,origin,nace_main,ind2,ind,urban_m,region_m,founded_date,labor_avg,status_alive,default
1,2005,6.934257e+06,13/05/2005,31/12/2005,803.703674,4.648148e+03,9.311111e+03,0.000000,0.000000,0.000000,...,NaN,2660.0,26.0,2.0,3.0,East,26/04/2005,NaN,1,1
10,2005,1.777654e+07,01/01/2005,31/12/2005,11722.222660,1.679889e+05,1.569148e+05,0.000000,1851.851807,1851.851807,...,NaN,2620.0,26.0,2.0,1.0,Central,01/07/1997,NaN,1,0
26,2005,1.012970e+08,01/01/2005,31/12/2005,8118.518555,8.058519e+04,4.419259e+04,0.000000,0.000000,0.000000,...,NaN,2620.0,26.0,2.0,3.0,East,01/11/1998,NaN,1,0
27,2005,1.108378e+08,01/01/2005,31/12/2005,54792.593750,7.259037e+05,2.000778e+05,59829.628910,15018.518550,-44811.109380,...,NaN,2611.0,26.0,2.0,3.0,East,05/12/1995,NaN,1,0
29,2005,1.295472e+08,01/01/2005,31/12/2005,2148.148193,4.185185e+03,6.120741e+04,0.000000,0.000000,0.000000,...,NaN,2670.0,26.0,2.0,3.0,West,30/11/1998,NaN,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19974,2015,3.917690e+11,01/01/2015,31/12/2015,759.259277,5.746296e+04,2.071852e+04,0.000000,0.000000,0.000000,...,NaN,2630.0,26.0,2.0,1.0,Central,26/12/1990,0.083333,1,0
20008,2015,4.104060e+11,01/01/2015,31/12/2015,22700.000000,3.223815e+05,8.598889e+04,7125.925781,859.259277,-6266.666504,...,NaN,2660.0,26.0,2.0,1.0,Central,17/12/1990,0.888889,1,0
20036,2015,4.244830e+11,01/01/2015,31/12/2015,259.259247,2.319259e+04,1.140741e+04,0.000000,0.000000,0.000000,...,NaN,2640.0,26.0,2.0,3.0,Central,15/10/2002,0.083333,1,0
20055,2015,4.352820e+11,01/01/2015,31/12/2015,NaN,4.362126e+06,4.198422e+06,27400.000000,118.518517,-27281.482420,...,NaN,2651.0,26.0,2.0,1.0,Central,03/01/1994,7.958333,1,0


In [74]:
holdout_set = work_df[(work_df.ind2 == 26) 
                      & (work_df.year == 2014) 
                      & (work_df.sales >= 1000) 
                      & (work_df.sales <= 10_000_000) 
]
holdout_set.shape

(1037, 42)

In [79]:
training_set = work_df[(work_df.ind2 == 26) 
                      & (work_df.year != 2014) 
                      & (work_df.sales >= 1000) 
                      & (work_df.sales <= 10_000_000) 
]
training_set.shape

(10727, 42)

In [75]:
holdout_set.default.sum()

56

In [80]:
training_set[(work_df.year == 2015)].default.sum()

489

In [81]:
training_set[(work_df.year == 2015)]['year'].value_counts()

year
2015    1038
Name: count, dtype: int64

In [87]:
holdout_set.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1037 entries, 16461 to 18289
Data columns (total 42 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   year                  1037 non-null   int64  
 1   comp_id               1037 non-null   float64
 2   begin                 1037 non-null   object 
 3   end                   1037 non-null   object 
 4   amort                 1035 non-null   float64
 5   curr_assets           1037 non-null   float64
 6   curr_liab             1037 non-null   float64
 7   extra_exp             1037 non-null   float64
 8   extra_inc             1037 non-null   float64
 9   extra_profit_loss     1037 non-null   float64
 10  fixed_assets          1037 non-null   float64
 11  inc_bef_tax           1037 non-null   float64
 12  intang_assets         1037 non-null   float64
 13  inventories           1037 non-null   float64
 14  liq_assets            1037 non-null   float64
 15  material_exp         